In [4]:
import pandas as pd
import numpy as np
import os

In [5]:
#Create path
path= r'/Users/shantamaraj/Desktop/INSTACART DATA/Prepared Data'

In [6]:
#Import pickle file from previous exercise
ords_prods_merge= pd.read_pickle(os.path.join(path, "ords_prods_merge_2.pkl"))

In [7]:
ords_prods_merge.head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,updated_busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,NaN,Regularly busy,Regularly busy,Average orders
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,NaN,Regularly busy,Regularly busy,Average orders
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,4.4,Low- range product,Regularly busy,Regularly busy,Average orders


In [8]:
#2 Aggregate mean of the “order_number” column grouped by “department_id”
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


#3 Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

Overall, the full dataframe shows higher average order numbers across most departments, which makes sense since it captures a wider range of customer behavior over time.

In [31]:
#4 CREATE a loyalty flag

# Step 1: Create 'max_order' per user using transform()
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

# Step 2: Use loc() to assign loyalty labels
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] >= 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] < 10, 'loyalty_flag'] = 'New customer'

/var/folders/9x/769d711n421bxzq6p4jzg37m0000gn/T/ipykernel_19548/3360641810.py:4: FutureWarning: The provided callable <function max at 0x10efb44a0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


In [32]:
#Check the output
ords_prods_merge.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,updated_busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,NaN,Regularly busy,Regularly busy,Average orders,10,Regular customer
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,NaN,Regularly busy,Regularly busy,Average orders,10,Regular customer
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,4.4,Low- range product,Regularly busy,Regularly busy,Average orders,10,Regular customer
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low- range product,Regularly busy,Regularly busy,Average orders,10,Regular customer
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low- range product,Regularly busy,Regularly busy,Average orders,10,Regular customer


In [34]:
#Check the value counts of the new "loyalty_flag" column
ords_prods_merge["loyalty_flag"].value_counts()

loyalty_flag
Regular customer    16664890
Loyal customer      10293737
New customer         5475585
Name: count, dtype: int64

In [35]:
#Regular customers make up the largest segment, suggesting a strong base of users with moderate but consistent ordering behavior.
#Loyal customers are a major group as it shows a large portion of users have placed many repeat orders 
#New customers are the smallest group

In [36]:
#5 Use the loyalty flag and check the basic statistics of the product prices for each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.388747  1.0  99999.0
New customer      13.208206  1.0  99999.0
Regular customer  12.561576  1.0  99999.0

In [38]:
ords_prods_merge[ords_prods_merge['prices'] == 99999.0]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,updated_busiest_days,busiest_period_of_day,max_order,loyalty_flag
129018,183964,873,3,0,10,7.0,33664,11,0,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Busiest day,Busiest days,Fewest orders,8,New customer
129031,1851256,873,4,6,12,13.0,33664,8,1,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Regularly busy,Regularly busy,Fewest orders,8,New customer
284516,1915696,1893,1,5,17,NaN,33664,10,0,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Regularly busy,Regularly busy,Average orders,6,New customer
284527,2763293,1893,2,4,16,13.0,33664,6,1,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Least busy,Slowest days,Fewest orders,6,New customer
284550,2564805,1893,4,1,17,30.0,33664,3,1,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Regularly busy,Busiest days,Average orders,6,New customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32109022,2249946,204099,29,0,8,4.0,33664,1,0,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Busiest day,Busiest days,Average orders,39,Regular customer
32109039,2363282,204099,31,0,9,2.0,33664,1,1,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Busiest day,Busiest days,Fewest orders,39,Regular customer
32154885,3181945,204395,13,3,15,8.0,33664,25,0,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Regularly busy,Slowest days,Fewest orders,15,Regular customer
32284308,2486215,205227,7,3,20,4.0,33664,8,0,2 % Reduced Fat Milk,84,16,99999.0,High-range product,Regularly busy,Slowest days,Average orders,12,Regular customer


#5 Loyal customers have the lowest average price 10.39 suggesting they may focus on lower cost or essential items
#New customers spend the most on average 13.21
#The max price is unusually high 99,999.00  for all groups, likely indicating a  data entry error.
Overall, loyal customers seem to favor more budget-friendly options, while newer or regular customers show slightly higher spending per item. This could inform marketing strategies—like offering loyalty rewards focused on value-based pricing.

In [43]:
#6 Create a Spending Flag 

#Calculate the mean price per user across all their orders
ords_prods_merge['avg_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

#Create the spending_flag based on the average price
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

/var/folders/9x/769d711n421bxzq6p4jzg37m0000gn/T/ipykernel_19548/718973409.py:4: FutureWarning: The provided callable <function mean at 0x10efb4ea0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)


In [54]:
#Check data output
ords_prods_merge['spending_flag'].value_counts()

spending_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64

In [55]:
#Calculate the median 'days_since_prior_order' per user
ords_prods_merge['median_days_since'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

#Apply the conditions using loc()
ords_prods_merge.loc[ords_prods_merge['median_days_since'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since'] > 10) & (ords_prods_merge['median_days_since'] <= 20), 'order_frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

/var/folders/9x/769d711n421bxzq6p4jzg37m0000gn/T/ipykernel_19548/3523527884.py:2: FutureWarning: The provided callable <function median at 0x10f0f0540> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [60]:
ords_prods_merge.head(10)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,updated_busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_days_since,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,NaN,Regularly busy,Regularly busy,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,NaN,Regularly busy,Regularly busy,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,...,Low- range product,Regularly busy,Regularly busy,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,...,Low- range product,Regularly busy,Regularly busy,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,Low- range product,Regularly busy,Regularly busy,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
5,2398795,1,2,3,7,15.0,196,1,1,Soda,...,NaN,Regularly busy,Slowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
6,2398795,1,2,3,7,15.0,10258,2,0,Pistachios,...,Low- range product,Regularly busy,Slowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
7,2398795,1,2,3,7,15.0,12427,3,1,Original Beef Jerky,...,Low- range product,Regularly busy,Slowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
8,2398795,1,2,3,7,15.0,13176,4,0,Bag of Organic Bananas,...,NaN,Regularly busy,Slowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer
9,2398795,1,2,3,7,15.0,26088,5,1,Aged White Cheddar Popcorn,...,Low- range product,Regularly busy,Slowest days,Average orders,10,Regular customer,6.367797,Low spender,20.5,Non-frequent customer


In [61]:
ords_prods_merge['order_frequency_flag'].value_counts()

order_frequency_flag
Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: count, dtype: int64

In [9]:
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [10]:
#Export as pickle
ords_prods_merge.to_pickle(os.path.join(path, 'orders_products_merged_grouped_4.8.pkl'))